In [52]:
from pdb import set_trace as debug
from pandas.io.parsers import read_csv
from IPython.parallel import Client,require
from collections import Counter

In [53]:
import ujson
import gzip
import datetime
import re
import csv
import gzip

In [54]:
#--> RAW_FILE_DIR with the directory that contains google_play__main.json.gz
#--> OUT_DIR with the directory of where you want the processed files to go
#--> DEBUG whether to print out created CSV files or not
#--> COMPRESS_LEVEL gzip compression level. warning: severely impacts runtime.
#--> LINE_LIMIT how many lines to iterate through in the raw file. For debugging.
DUMP_DATE = '2015_04_07_23_12'
RAW_FILE_DIR = '/Users/catherinaxu/Desktop/Stanford/EC/MIG/MixRank/MixRank Raw'
OUT_DIR = '/Users/catherinaxu/Desktop/Stanford/EC/MIG/MixRank/MixRank Out'
DEBUG = 1
COMPRESS_LEVEL = 1
LINE_LIMIT = -1 # -1 means run for the entire file
PARALLEL = 0

In [55]:
RAW_FILE_IOSSDKSINDEX = '{}/mixrank__ios_sdks_index.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFOTEST2 = '{}/mixrank__ios_sdks_info_test2.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_ANDROIDTOP500 = '{}/mixrank__android_top_500.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSTOP200 = '{}/mixrank__ios_top_200.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFOFULL = '{}/mixrank__ios_sdks_info_full.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFOTEST = '{}/mixrank__ios_sdks_info_test.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_ANDROIDSDKSINFO = '{}/mixrank__android_sdks_info.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSINFO = '{}/mixrank__ios_sdks_info.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_ANDROIDSDKSPERAPP = '{}/mixrank__android_sdks_per_app.json.gz'.format(RAW_FILE_DIR)
RAW_FILE_IOSSDKSPERAPP = '{}/mixrank__ios_sdks_per_app.json.gz'.format(RAW_FILE_DIR)
OUT_BASE = OUT_DIR+'/{}__'+DUMP_DATE+'.csv.gz'

In [56]:
def iter_json_gzip(filename,LINE_LIMIT=LINE_LIMIT,no_verify_appid_time=1):
    with gzip.open(filename,'rt') as file_iter:
        c = 0 
        #debug 
        #file_content = file_iter.read()
        #print (file_content)
        
        #iterating through lines in file
        for line in file_iter: 
            c += 1
            
            #makes sure that it does not read over the line limit
            if c > LINE_LIMIT and LINE_LIMIT > 0:
                break
                
            if isinstance(line,str):
                if len(line)>0:        
                    out = ujson.loads(line)
                    
                    #if it is the proper format
                    if isinstance(out,dict):
                        if 'app_id' in out and 'timestamp' in out \
                        or no_verify_appid_time:
                            yield out

# IOSSDKSINDEX

In [57]:
# The gzip file contains the same list twice
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def IOSSDKSINDEX(file_in, out_base, compress = 1):
    headers = ['sdk_id']
    out_filename = out_base.format('IOSSDKSINDEX')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json.get('sdk_id')])
    return out_filename

In [58]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSSDKSINDEX(RAW_FILE_IOSSDKSINDEX,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,sdk_id
0,atvalidations
1,revmob
2,cocoapods
3,asyncimageview
4,urban-airship
5,nend
6,atfunctionalcollectionextensions
7,ziparchive
8,chartboost
9,cocosdenshion


#IOSSDKSINFOTEST2

In [59]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def IOSSDKSINFOTEST2(file_in, out_base, compress = 1):
    headers = ['app_id', 'sdk_list', 'rank'] 
    out_filename = out_base.format('IOSSDKSINFOTEST2')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json['app_id'], json.get('sdk_list'), json.get('rank_highest')])
    return out_filename

In [60]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSSDKSINFOTEST2(RAW_FILE_IOSSDKSINFOTEST2,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_list,rank
0,301259483,"afscrollview,appirater,aqgridview,crashlytics,...",NaN
1,487758420,"asihttprequest,cocos2d,cocosdenshion,flurry,fo...",NaN
2,361960662,"facebook,oauth,revmob,sbjson-for-objective-c",NaN
3,333289016,touchxml,NaN
4,416454511,"admob,applovin,appsfire,apsalar,asihttprequest...",NaN
5,341311361,"pushwoosh,asihttprequest,crashlytics,facebook,...",NaN
6,489552307,"appirater,aqgridview,asihttprequest,icarousel,...",NaN
7,381260755,"asihttprequest,cocoaasyncsocket,dcroundswitch,...",NaN
8,363484920,"admob,cocoaasyncsocket,dart-for-publishers,reg...",NaN
9,355833469,"jlroutes,analytics,bugsnag,cocoapods,reactivec...",NaN


#ANDROIDTOP500

In [61]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def ANDROIDTOP500(file_in, out_base, compress = 1):
    headers = ['category', 'sdk_count', 'updated', 'sdk_list', 'rating_average', 'app_id'] 
    out_filename = out_base.format('ANDROIDTOP500')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json['Category'], json['Num SDKs'], json['Updated Date'], json['sdks'], json['Star Rating'], json['Package Name']])
    return out_filename

In [62]:
df_debug = None
if DEBUG:
    df_debug = read_csv(ANDROIDTOP500(RAW_FILE_ANDROIDTOP500,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,category,sdk_count,updated,sdk_list,rating_average,app_id
0,Strategy,20,2014-07-02,Android Internal;Google Analytics;Google Play ...,4.6,com.supercell.clashofclans
1,Casual,21,2014-09-02,AdMob;AdMob Mediation;Android API;Google Autho...,4.4,com.king.candycrushsaga
2,Strategy,21,2014-08-05,AdMob;Amazon GameCircle;Android Support Librar...,4.2,com.machinezone.gow
3,Casual,21,2014-09-04,AdMob;AdMob Mediation;Android API;Google Autho...,4.3,com.king.farmheroessaga
4,Family,20,2014-07-01,Android Internal;Google Analytics;Google Play ...,4.4,com.supercell.hayday
5,Casual,18,2014-09-04,AdMob;AdMob Mediation;Amazon GameCircle;Androi...,4.3,com.midasplayer.apps.bubblewitchsaga2
6,NaN,NaN,NaN,NaN,NaN,com.glu.baseball
7,Casual,20,2014-08-28,AdMob;AdMob Mediation;Android API;Google Autho...,4.2,com.king.petrescuesaga
8,Casino,22,2014-08-28,HockeyApp;Android Support Library;Apache Commo...,4.4,air.com.playtika.slotomania
9,Strategy,24,2014-08-06,AdMob;AdMob Mediation;Android API;Google Analy...,4.6,com.igg.castleclash


#IOSTOP200 

In [63]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def IOSTOP200(file_in, out_base, compress = 1):
    headers = ['app_id', 'price', 'sdk_count', 'rank_highest', 'release_date', 'sdk_list'] 
    out_filename = out_base.format('IOSTOP200')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json['App ID'], json['Price'], json['Number of SDKs'], json['Best Rank'], json['Release Date'], json['sdks']])
    return out_filename

In [64]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSTOP200(RAW_FILE_IOSTOP200,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,price,sdk_count,rank_highest,release_date,sdk_list
0,529479190,0,10,1,2012-06-14,AdColony;Appirater;ATFunctionalCollectionExten...
1,284035177,0,9,1,2008-07-11,AdMarvel;comScore (Digital Analytix);Crashlyti...
2,553834731,0,4,1,2012-10-19,NaN
3,667728512,0,NaN,1,2013-07-25,NaN
4,608206510,0,5,1,2013-12-11,NaN
5,506627515,0,12,1,2012-05-04,AdColony;Appirater;ATFunctionalCollectionExten...
6,349160522,0,19,1,2010-01-12,GoogleConversionTracking;Amazon Web Services (...
7,324684580,0,NaN,1,2009-09-07,NaN
8,860822992,0,17,1,2014-06-02,NaN
9,305939712,0,10,2,2009-03-27,HasOffers (Mobile App Tracking);RBStoryboardLi...


#IOSSDKSINFOFULL

In [65]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def IOSSDKSINFOFULL(file_in, out_base, compress = 1):
    headers = ['app_id', 'sdk_list', 'rank_highest'] 
    out_filename = out_base.format('IOSSDKSINFOFULL')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json['app_id'], json['sdk_list'], json['highest_rank']])
    return out_filename

In [66]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSSDKSINFOFULL(RAW_FILE_IOSSDKSINFOFULL,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_list,rank_highest
0,452771538,"asihttprequest,facebook,jsonkit,mkstorekit,oau...",-1
1,379892783,"aarki,appirater,cocoalumberjack,cocos2d,cocosd...",-1
2,409978794,"parse,uikit",-1
3,507225541,asihttprequest,-1
4,452236050,brightcove-video-cloud-app-sdk-player-and-shar...,-1
5,385528125,"facebook,oauth,revmob,sbjson-for-objective-c",-1
6,344010778,"admob,dart-for-publishers,uikit",-1
7,394354689,"adcolony,admob,asihttprequest,atfunctionalcoll...",-1
8,470906595,"fmdb,pstcollectionview,ssziparchive,svprogress...",-1
9,377869410,"admob,dacircularprogress,dart-for-publishers,i...",-1


#IOSSDKSINFOTEST

In [67]:
@require(iter_json_gzip,'ujson','gzip','csv','datetime')
def IOSSDKSINFOTEST(file_in, out_base, compress = 1):
    headers = ['rank_highest', 'app_id', 'sdk_list'] 
    out_filename = out_base.format('IOSSDKSINFOTEST')
    with gzip.open(out_filename, 'wt', compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        for json in iter_json_gzip(file_in):
            csv_writer.writerow([json['highest_rank'], json['app_id'], json['sdk_list']])
    return out_filename

In [68]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSSDKSINFOTEST(RAW_FILE_IOSSDKSINFOTEST,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,rank_highest,app_id,sdk_list
0,-1,301259483,"afscrollview,appirater,aqgridview,crashlytics,..."
1,-1,487758420,"asihttprequest,cocos2d,cocosdenshion,flurry,fo..."
2,-1,361960662,"facebook,oauth,revmob,sbjson-for-objective-c"
3,-1,333289016,touchxml
4,-1,416454511,"admob,applovin,appsfire,apsalar,asihttprequest..."
5,-1,341311361,"pushwoosh,asihttprequest,crashlytics,facebook,..."
6,-1,489552307,"appirater,aqgridview,asihttprequest,icarousel,..."
7,-1,381260755,"asihttprequest,cocoaasyncsocket,dcroundswitch,..."
8,-1,363484920,"admob,cocoaasyncsocket,dart-for-publishers,reg..."
9,-1,355833469,"jlroutes,analytics,bugsnag,cocoapods,reactivec..."


#ANDROIDSDKSINFO

In [69]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def ANDROIDSDKSINFO(file_in,out_base,compress=1):
    headers = ['sdk_id','app_id', 'release_date']
    out_filename = out_base.format('ANDROIDSDKSINFO')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            sdk_id = json.get('sdk_id');
            
            #if there are any apps that have installed this sdk
            if ('installs' in json and json.get('installs')):
                # get each of the app names and write a new row

                # why does offset 0 work for everything? but not offset 200? 
                # looping through each of the results
                for each_app in json['installs']['offset_0']['results']:
                    obs = (sdk_id, each_app.get('pname'), each_app.get('upload_date'))
                    csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (sdk_id, "", "")
                csv_writer.writerow(obs)
    return out_filename


In [70]:
df_debug = None
if DEBUG:
    df_debug = read_csv(ANDROIDSDKSINFO(RAW_FILE_ANDROIDSDKSINFO,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,sdk_id,app_id,release_date
0,zxing-zebra-crossing-,NaN,NaN
1,xml-pull-parsing,NaN,NaN
2,ksoap2,NaN,NaN
3,android-calendarview,NaN,NaN
4,apache-cordova,jp.naver.line.android,2014-07-17
5,apache-cordova,vStudio.Android.Camera360,2014-06-25
6,apache-cordova,com.amazon.mShop.android,2014-07-16
7,apache-cordova,com.unityconceptapps.tcr.kiat,2014-07-10
8,apache-cordova,com.usablenet.mobile.walgreen,2014-06-27
9,apache-cordova,org.wikipedia,2014-06-25


#IOSSDKSINFO

In [71]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def IOSSDKSINFO(file_in,out_base,compress=1):
    headers = ['sdk_id','app_id', 'release_date']
    out_filename = out_base.format('IOSSDKSINFO')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            sdk_id = json.get('sdk_id');
            
            #if there are any apps that have installed this sdk
            if ('installs' in json and json.get('installs')):
                # get each of the app names and write a new row

                # why does offset 0 work for everything? but not offset 200? 
                # looping through each of the results
                for each_app in json['installs']['offset_0']['results']:
                    obs = (sdk_id, each_app.get('id'), each_app.get('upload_date'))
                    csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (sdk_id, "", "")
                csv_writer.writerow(obs)
    return out_filename

In [72]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSSDKSINFO(RAW_FILE_IOSSDKSINFO,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,sdk_id,app_id,release_date
0,atvalidations,NaN,NaN
1,revmob,NaN,NaN
2,cocoapods,NaN,NaN
3,asyncimageview,NaN,NaN
4,urban-airship,NaN,NaN
5,nend,NaN,NaN
6,atfunctionalcollectionextensions,420009108,2011-08-04
7,atfunctionalcollectionextensions,469369175,2012-06-28
8,atfunctionalcollectionextensions,573511269,2012-11-05
9,atfunctionalcollectionextensions,512939461,2012-05-24


#ANDROIDSDKSPERAPP

In [73]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def ANDROIDSDKSPERAPP(file_in,out_base,compress=1):
    headers = ['app_id','sdk_id']
    out_filename = out_base.format('ANDROIDSDKSPERAPP')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            app_id = json.get('app_id');
            
            #if there are any apps that have installed this sdk
            if ('sdks' in json and json.get('sdks')):
                # get each of the app names and write a new row

                # why does offset 0 work for everything? but not offset 200? 
                # looping through each of the results
                for each_app in json['sdks']['offset_0']['results']:
                    obs = (app_id, each_app.get('slug'))
                    csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (app_id, "")
                csv_writer.writerow(obs)
    return out_filename

In [74]:
df_debug = None
if DEBUG:
    df_debug = read_csv(ANDROIDSDKSPERAPP(RAW_FILE_ANDROIDSDKSPERAPP,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_id
0,com.themesfordroid.spacethemegolauncherex,android-api
1,com.themesfordroid.spacethemegolauncherex,google-authorization
2,com.themesfordroid.spacethemegolauncherex,google-cloud-messaging
3,com.themesfordroid.spacethemegolauncherex,google-play-game-services
4,com.themesfordroid.spacethemegolauncherex,google-play-location-apis
5,com.themesfordroid.spacethemegolauncherex,google-play-maps
6,com.themesfordroid.spacethemegolauncherex,google-play-panorama
7,com.themesfordroid.spacethemegolauncherex,google-play-services
8,com.themesfordroid.spacethemegolauncherex,google-plus
9,com.themesfordroid.spacethemegolauncherex,google-wallet


#IOSSDKSPERAPP

In [75]:
@require(iter_json_gzip,gzip,datetime,re,csv)
def IOSSDKSPERAPP(file_in,out_base,compress=1):
    headers = ['app_id','sdk_id']
    out_filename = out_base.format('IOSSDKSPERAPP')
    with gzip.open(out_filename,'wt',compress) as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(headers)
        
        # for each observation, in this case "overview"
        for json in iter_json_gzip(file_in):
            
            #obtain and record the sdk_id, which is crucial to app ID
            app_id = json.get('app_id');
            
            #if there are any apps that have installed this sdk
            if ('sdks' in json and json.get('sdks')):
                # get each of the app names and write a new row

                # why does offset 0 work for everything? but not offset 200? 
                # looping through each of the results
                for each_app in json['sdks']['offset_0']['results']:
                    obs = (app_id, each_app.get('slug'))
                    csv_writer.writerow(obs)
                    
            #otherwise just record the sdk ID
            else:
                obs = (app_id, "")
                csv_writer.writerow(obs)
    return out_filename

In [76]:
df_debug = None
if DEBUG:
    df_debug = read_csv(IOSSDKSPERAPP(RAW_FILE_IOSSDKSPERAPP,OUT_BASE,COMPRESS_LEVEL),compression='gzip',header=0)
df_debug

,app_id,sdk_id
0,301259483,afscrollview
1,301259483,appirater
2,301259483,aqgridview
3,301259483,crashlytics
4,301259483,facebook
5,301259483,flurry
6,301259483,google-analytics
7,301259483,googleconversiontracking
8,301259483,google-data-apis-gdata-
9,301259483,hasoffers-mobile-app-tracking-
